In [40]:
!pip install spacy
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl
    

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)


In [41]:
import pandas as pd
import spacy

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [42]:
script = pd.read_csv('Game_of_Thrones_Script_count_senti.csv')
script.head()

,Release Date,Season,Episode,Episode Title,Name,Sentence,Tokenize Words,Tokenize Words Alphanumeric Only,Sentence Word Count,Row,Family Name,Sentiment Scores,Polarity,Subjectivity
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...,"['What', 'do', 'you', 'expect', '?', 'They', ""...","['What', 'do', 'you', 'expect', 'They', 'savag...",25,0,other,"Sentiment(polarity=-0.125, subjectivity=0.375)",-0.125,0.375
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...,"['I', ""'ve"", 'never', 'seen', 'wildlings', 'do...","['I', 'never', 'seen', 'wildlings', 'do', 'a',...",21,1,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?,"['How', 'close', 'did', 'you', 'get', '?']","['How', 'close', 'did', 'you', 'get']",5,2,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
3,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.,"['Close', 'as', 'any', 'man', 'would', '.']","['Close', 'as', 'any', 'man', 'would']",5,3,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000
4,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.,"['We', 'should', 'head', 'back', 'to', 'the', ...","['We', 'should', 'head', 'back', 'to', 'the', ...",7,4,other,"Sentiment(polarity=0.0, subjectivity=0.0)",0.000,0.000


In [43]:
top_20 = script.groupby(['Name', 'Family Name'])[['Sentence Word Count']].sum().reset_index().sort_values(by=['Sentence Word Count']
                                                                                         , ascending=[0])

In [44]:
script_top5 = script[script["Name"].isin(top_20["Name"][:5])]

/var/folders/ll/tlyhkxyx7qj42_ffkhwd31z00000gn/T/ipykernel_14140/557377519.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  script_top5 = script[script["Name"].isin(top_20["Name"][:5])]


In [45]:
script_top_20 = script[script["Name"].isin(top_20["Name"][:20])]

/var/folders/ll/tlyhkxyx7qj42_ffkhwd31z00000gn/T/ipykernel_14140/3396793553.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  script_top_20 = script[script["Name"].isin(top_20["Name"][:20])]


In [46]:
script_top_20 = script_top_20[["Sentence", "Name"]]

In [47]:
script_top_20.columns = ["Sentence", "Label"]

In [48]:
script_top_20.head()

,Sentence,Label
15,Go on. Father's watching.,jon snow
16,And your mother.,jon snow
18,Thank you.,sansa stark
21,"Don't think too much, Bran.",jon snow
22,Relax your bow arm.,robb stark


In [49]:
script_top_20.to_csv('Game_of_Thrones_Script_top20.csv', encoding='utf-8', index=False)

In [50]:
script_top5["Name"].unique()

array(['jon snow', 'jaime lannister', 'cersei lannister',
       'tyrion lannister', 'daenerys targaryen'], dtype=object)

In [51]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [52]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [53]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))

In [54]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [55]:
from sklearn.model_selection import train_test_split

X = script_top5['Sentence'] # the features we want to analyze
ylabels = script_top5['Name'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [56]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()


# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))


In [ ]:
pipe.classes_

In [ ]:
pipe.predict_proba(["Yes, and the dwarf loves his whine and the women"])

In [ ]:
pipe.predict_proba(["The high sparrow annoys me."])

In [ ]:
pipe.predict_proba(["I own the dragons."])

In [ ]:
pipe.predict_proba(["I can't fight anymore because I lost my hand."])

In [ ]:
pipe.predict_proba(["My father is Eddard Stark."])

In [ ]:
pipe.predict_proba(["My father is Tywin Lannister."])

In [ ]:
pipe.predict_proba(["The north remembers."])

The last model was generic. In the next step I will try to tune the model a little and use Season 1 - 7 for train and cross validation. Then let's see how well this model performs on Season 8.

In [ ]:
X = script_top5[script_top5["Season"] != "Season 8"]['Sentence'] # the features we want to analyze
ylabels = script_top5[script_top5["Season"] != "Season 8"]['Name'] # the labels, or answers, we want to test against

X_holdout = script_top5[script_top5["Season"] == "Season 8"]['Sentence']#
ylabels_holdout = script_top5[script_top5["Season"] == "Season 8"]['Name']

In [ ]:
# Logistic Regression Classifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

hyperparams = {'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], "classifier__solver": ["newton-cg", "liblinear"]}

# Create pipeline using Bag of Words
pipe2 = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])


grid = GridSearchCV(pipe2, param_grid = hyperparams, cv=10)

# model generation
grid.fit(X,ylabels)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
best_model = grid.best_estimator_

In [ ]:
from sklearn import metrics

# Predicting with my holdout dataset (Season 8)
predictions = best_model.predict(X_holdout)
print("Logistic Regression Accuracy:",metrics.accuracy_score(ylabels_holdout, predictions))

In [ ]:
best_model.classes_

In [ ]:
best_model.predict_proba(["Yes, and the dwarf loves his whine and the women"])

In [ ]:
best_model.predict_proba(["The love for my children is above everything"])

In [ ]:
best_model.predict_proba(["I will break the chains and free millions of people."])

In [ ]:
best_model.predict_proba(["I won't be able to fight ever again..."])

In [ ]:
best_model.predict_proba(["My father is Tywin Lannister"])